In [4]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import cv2
import os
import random
import time
import tensorflow as tf

In [5]:
n_train = 4640
n_valid = 131

In [6]:
IMG_SIZE = 160

def load(dir, n):
    x_data, y_data = [], []
    for i in range(n):
        image = cv2.imread(dir + "/" + str(i + 1) + ".png")
        r = open(dir + "/" + str(i + 1) + ".txt", "r")
        label = int(r.readline())
        image = tf.cast(image, tf.float32)
        image = (image/127.5) - 1
        x_data.append(image)
        y_data.append(label)
    return x_data, y_data

X_train, Y_train = load("/home/dima/PycharmProjects/Practics/Dataset/train", n_train)
X_valid, Y_valid = load("/home/dima/PycharmProjects/Practics/Dataset/valid", n_valid)

Y_train = tf.cast(Y_train, tf.int32)
Y_valid = tf.cast(Y_valid, tf.int32)
Y_train -= 1
Y_valid -= 1
cnt_train = [0, 0, 0, 0]
cnt_valid = [0, 0, 0, 0]
for i in range(n_train):
    cnt_train[Y_train[i]] += 1
for i in range(n_valid):
    cnt_valid[Y_valid[i]] += 1
print(cnt_train)
print(cnt_valid)
X_train_norm = tf.norm(X_train)
X_valid_norm = tf.norm(X_valid)

#X_train = X_train / X_train_norm
#X_valid = X_valid / X_valid_norm


[1248, 1184, 1200, 1008]
[34, 34, 31, 32]


In [7]:
train = tf.data.Dataset.from_tensor_slices( (X_train, Y_train) )
valid = tf.data.Dataset.from_tensor_slices( (X_valid, Y_valid) )

In [8]:
print(train)
print(valid)

<TensorSliceDataset shapes: ((160, 160, 3), ()), types: (tf.float32, tf.int32)>
<TensorSliceDataset shapes: ((160, 160, 3), ()), types: (tf.float32, tf.int32)>


In [9]:
BATCH_SIZE = 29
SHUFFLE_BUFFER_SIZE = 1000

In [10]:
train_batches = train.batch(BATCH_SIZE)
validation_batches = valid.batch(BATCH_SIZE)
print(train_batches, validation_batches)

<BatchDataset shapes: ((None, 160, 160, 3), (None,)), types: (tf.float32, tf.int32)> <BatchDataset shapes: ((None, 160, 160, 3), (None,)), types: (tf.float32, tf.int32)>


In [11]:
for image_batch, label_batch in train_batches.take(1):
    pass

image_batch.shape

TensorShape([29, 160, 160, 3])

In [12]:
IMG_SHAPE = (IMG_SIZE, IMG_SIZE, 3)


base_model = tf.keras.applications.MobileNetV2(input_shape=IMG_SHAPE,
                                               include_top=False,
                                               weights='imagenet')


In [13]:
feature_batch = base_model(image_batch)
print(feature_batch.shape)


(29, 5, 5, 1280)


In [14]:
base_model.trainable = False
base_model.summary()


Model: "mobilenetv2_1.00_160"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 160, 160, 3) 0                                            
__________________________________________________________________________________________________
Conv1_pad (ZeroPadding2D)       (None, 161, 161, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
Conv1 (Conv2D)                  (None, 80, 80, 32)   864         Conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_Conv1 (BatchNormalization)   (None, 80, 80, 32)   128         Conv1[0][0]                      
_______________________________________________________________________________

In [15]:
global_average_layer = tf.keras.layers.GlobalAveragePooling2D()
feature_batch_average = global_average_layer(feature_batch)
print(feature_batch_average.shape)


(29, 1280)


In [16]:
import tensorflow.keras as keras

prediction_layer = keras.layers.Dense(4, activation = 'softmax')
prediction_batch = prediction_layer(feature_batch_average)
print(prediction_batch.shape)


(29, 4)


In [17]:
model = tf.keras.Sequential([
  base_model,
  global_average_layer,
  prediction_layer
])
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
mobilenetv2_1.00_160 (Model) (None, 5, 5, 1280)        2257984   
_________________________________________________________________
global_average_pooling2d (Gl (None, 1280)              0         
_________________________________________________________________
dense (Dense)                (None, 4)                 5124      
Total params: 2,263,108
Trainable params: 5,124
Non-trainable params: 2,257,984
_________________________________________________________________


In [18]:
base_learning_rate = 0.0001
model.compile(optimizer=tf.keras.optimizers.RMSprop(lr=base_learning_rate),
              loss='categorical_crossentropy',
              metrics=['accuracy'])


In [19]:
print(len(model.trainable_variables))
for image_batch, label_batch in validation_batches:
    print(label_batch)

2
tf.Tensor([1 1 3 3 0 3 3 2 1 2 2 1 1 2 1 3 0 1 2 2 3 3 3 0 3 2 2 3 1], shape=(29,), dtype=int32)
tf.Tensor([0 1 1 3 0 1 2 3 3 0 1 1 3 2 1 1 0 2 0 1 1 1 0 1 2 1 0 0 0], shape=(29,), dtype=int32)
tf.Tensor([1 0 2 3 3 0 0 0 3 0 2 0 0 1 1 3 1 1 3 0 1 1 2 0 0 3 2 0 2], shape=(29,), dtype=int32)
tf.Tensor([3 2 2 0 0 2 1 0 0 0 3 1 2 0 3 2 0 0 2 0 3 1 2 2 1 3 2 3 0], shape=(29,), dtype=int32)
tf.Tensor([0 2 3 2 3 1 2 3 2 1 3 3 1 3 2], shape=(15,), dtype=int32)


In [20]:
validation_steps = 5
initial_epochs = 10
model.evaluate(validation_batches, steps = validation_steps)
predictions = model.predict(validation_batches, steps = validation_steps, verbose = 1)
print(predictions)

5/5 [==============================] - 4s 863ms/step
[[0.09857979 0.03692968 0.21246202 0.65202856]
 [0.47112522 0.07051621 0.37175086 0.08660759]
 [0.23310466 0.00886274 0.03681368 0.7212189 ]
 [0.16641802 0.02252529 0.07012309 0.7409336 ]
 [0.24495046 0.04358891 0.05951136 0.6519493 ]
 [0.26747194 0.05841046 0.13047858 0.543639  ]
 [0.08285885 0.01510361 0.16979745 0.7322401 ]
 [0.08195275 0.00775322 0.14059114 0.7697029 ]
 [0.148809   0.0154463  0.22992322 0.6058215 ]
 [0.12603386 0.02568987 0.09206149 0.75621486]
 [0.0110202  0.01962591 0.1553906  0.8139633 ]
 [0.21658683 0.02389617 0.18736173 0.5721553 ]
 [0.2345487  0.02646334 0.04275668 0.6962313 ]
 [0.01464997 0.00269243 0.14661065 0.836047  ]
 [0.61421037 0.01330871 0.02462137 0.34785956]
 [0.03020283 0.02532776 0.31359163 0.6308778 ]
 [0.02235027 0.0039085  0.03854885 0.93519235]
 [0.06478381 0.09330947 0.25881758 0.5830891 ]
 [0.02908912 0.02361905 0.1369612  0.8103307 ]
 [0.15428129 0.00938533 0.08522624 0.75110716]
 [0.182

In [21]:
history = model.fit(train_batches,
                    epochs=initial_epochs,
                    validation_data=validation_batches)


Epoch 1/10
160/160 [==============================] - 196s 1s/step - loss: 9.3066 - accuracy: 0.2435 - val_loss: 0.0000e+00 - val_accuracy: 0.0000e+00
Epoch 2/10
160/160 [==============================] - 190s 1s/step - loss: 9.0937 - accuracy: 0.2638 - val_loss: 12.5710 - val_accuracy: 0.0992
Epoch 3/10
160/160 [==============================] - 195s 1s/step - loss: 9.0134 - accuracy: 0.2705 - val_loss: 12.4300 - val_accuracy: 0.0992
Epoch 4/10
160/160 [==============================] - 191s 1s/step - loss: 8.9686 - accuracy: 0.2703 - val_loss: 12.3231 - val_accuracy: 0.0992
Epoch 5/10
160/160 [==============================] - 190s 1s/step - loss: 8.9424 - accuracy: 0.2666 - val_loss: 12.2429 - val_accuracy: 0.0992
Epoch 6/10
160/160 [==============================] - 191s 1s/step - loss: 8.9256 - accuracy: 0.2659 - val_loss: 12.1823 - val_accuracy: 0.0992
Epoch 7/10
160/160 [==============================] - 192s 1s/step - loss: 8.9136 - accuracy: 0.2644 - val_loss: 12.1356 - val_ac

In [22]:
model.evaluate(validation_batches, steps = validation_steps)
model.evaluate(train_batches, steps = 160)

160/160 [==============================] - 113s 707ms/step - loss: 10.8708 - accuracy: 0.3629


[10.870815688371659, 0.36293104]

In [23]:
predictions = model.predict(validation_batches, steps = validation_steps, verbose = 1)
print(predictions)

5/5 [==============================] - 3s 651ms/step
[[0.10401601 0.09308014 0.15427086 0.648633  ]
 [0.48468465 0.16557    0.2636854  0.08605993]
 [0.23443174 0.02491697 0.02572087 0.7149304 ]
 [0.17215565 0.05203791 0.05344416 0.7223623 ]
 [0.24060436 0.10328309 0.04197953 0.61413294]
 [0.26303995 0.13095246 0.08980057 0.51620704]
 [0.08955643 0.03854449 0.1280578  0.74384123]
 [0.08940959 0.02234087 0.10580938 0.7824401 ]
 [0.16047639 0.03444273 0.18913716 0.61594373]
 [0.128543   0.06741311 0.06335872 0.7406852 ]
 [0.01206576 0.04924747 0.11885545 0.8198313 ]
 [0.22636421 0.05905604 0.14546978 0.56911   ]
 [0.23183347 0.07202992 0.02871433 0.66742224]
 [0.01770986 0.00692923 0.11512303 0.8602379 ]
 [0.5996689  0.0351913  0.01728275 0.347857  ]
 [0.03493974 0.07451005 0.23389092 0.6566593 ]
 [0.02571771 0.00956714 0.03092531 0.9337899 ]
 [0.06399361 0.23592742 0.17354307 0.5265359 ]
 [0.03190541 0.05246645 0.10588007 0.8097481 ]
 [0.16104855 0.02659386 0.05961689 0.75274074]
 [0.195